# Monitoring Logging Coupes in Victoria

## Set up your workspace

In [ ]:
import datacube
import geopandas as gpd
import pandas as pd

from odc.algo import to_f32, xr_geomedian, int_geomedian
from datacube.utils.geometry import Geometry, CRS

from dea_tools.datahandling import load_ard
from dea_tools.plotting import rgb
from dea_tools.spatial import xr_rasterize

pd.set_option('display.max_columns', None)

## Load spatial file with coupes

In [ ]:
log_season_gdf = gpd.read_file("data/LOG_SEASON_FILTERED.gpkg")

## Initial exploration

In [ ]:
log_season_gdf.head()

## Analyse logging event from 2019

In [ ]:
# Look for an event in 2019 to analyse

logging_events_2019 = log_season_gdf.loc[log_season_gdf.loc[:, "STARTYEAR_STRING"]=="2019", :].reset_index(drop=True)

In [ ]:
event = logging_events_2019.iloc[[1], :]

event

### Extract satellite imagery

In [ ]:
dc = datacube.Datacube(app='Logging_analysis')

In [ ]:
time_range = ("2018", "2021")

products = ["s2a_ard_granule", "s2b_ard_granule"]
measurements = ["nbart_red", "nbart_green", "nbart_blue", "nbart_nir_1"]
resolution = (10, -10)
output_crs = "EPSG:3577"
min_gooddata=0.99

geometry = Geometry(geom=event.geometry.item(), crs=logging_events_2019.crs)

In [ ]:
query = {
    "time": time_range,
    "measurements": measurements,
    "min_gooddata": min_gooddata,
    "resolution": resolution,
    "output_crs": "EPSG:3577",
    "geopolygon": geometry,
    "group_by": "solar_day"
}

In [ ]:
ds = load_ard(dc=dc,
              products=["s2a_ard_granule", "s2b_ard_granule"],
              **query)

# Generate a polygon mask to keep only data within the polygon
mask = xr_rasterize(event, ds)

# Mask dataset to set pixels outside the polygon to `NaN`
ds = ds.where(mask)

ds

In [ ]:
# Group by 2 month intervals
grouped = ds.resample(time='2MS')

In [ ]:
geomedian_grouped = grouped.map(xr_geomedian)

In [ ]:
rgb(geomedian_grouped, col="time")

In [ ]:
geomedian_grouped["NDVI"] = (geomedian_grouped.nbart_nir_1 - geomedian_grouped.nbart_red) / (geomedian_grouped.nbart_nir_1 + geomedian_grouped.nbart_red)

In [ ]:
geomedian_grouped["NDVI"].plot(col="time", col_wrap=4)

In [ ]:
geomedian_grouped["NDVI"].mean(dim=["x", "y"]).plot()